# GatorTronS

## Train the model

In [ ]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere

%run /home/ec2-user/SageMaker/LLM-NER-clinical-text/src/models/train_model.py \
--model_name 'UFNLP/gatortrons' \
--data_dir '/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/private/n2c2-2018/ner/' \
--batch_size 4 \
--num_train_epochs 5 \
--learning_rate 5e-5 \
--weight_decay 0.01 \
--new_model_dir "/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/n2c2-2018/gatortrons/" \
--path_umls_semtype '/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/SemGroups_2018.txt'


Loading and preprocessing the dataset ...
/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/private/n2c2-2018/ner/


Map:   0%|          | 0/1761 [00:00<?, ? examples/s]

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

Map:   0%|          | 0/1442 [00:00<?, ? examples/s]

Map:   0%|          | 0/1761 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1442 [00:00<?, ? examples/s]

The device to run the model: cuda
Load the pretrained model ...


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MegatronBertForTokenClassification were not initialized from the model checkpoint at UFNLP/gatortrons and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model has 354.237459millions parameters.


Epoch,Training Loss,Validation Loss


## Evaluate on test data


In [1]:
from src.models.model import *

# Load the model
model_loader = ModelLoader('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/n2c2-2018/gatortrons/')

# Evaluate the models on metrics
scores = model_loader.evaluate_model(dataset_name='/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/private/n2c2-2018/ner/',\
                                     path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt',\
                                     metric_names=['f1', 'precision', 'recall', 'matthews_correlation'])

scores

Loading and preprocessing the dataset ...
ncbi_disease


Map:   0%|          | 0/941 [00:00<?, ? examples/s]

{'f1': 0.9230959441861525,
 'precision': 0.8998375309216448,
 'recall': 0.948772382840148,
 'matthews_correlation': 0.8978492834665438}

## Fix the id2label in config file

In [1]:
import json
from src.data.data_loader import *
from huggingface_hub import HfApi

# Load the config file
f = open('../models/ncbi-disease/gatortrons/config.json')
config = json.load(f)

# Load the dataset with label mapping
dataset_loader = DatasetLoader(dataset_name='/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/private/n2c2-2018/ner/',\
                               model_name='/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/n2c2-2018/gatortrons/', \
                               path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt')
dataset, classmap, umls_label_code, _ = dataset_loader.load_dataset()

# Add the label to the config
id_to_label = {ind: classmap.int2str(ind) for ind in range(len(classmap.names))}
config['id2label'] = id_to_label
config['label2id'] = {id_to_label[key]: key for key in id_to_label.keys()}

# Save the config
with open('../models/n2c2-2018/gatortrons/config.json', 'w') as f:
    json.dump(config, f)


Loading and preprocessing the dataset ...
ncbi_disease


In [3]:
# Push the config to hub
api = HfApi()
api.upload_file(
    path_or_fileobj="../models/ncbi-disease/gatortrons/config.json",
    path_in_repo="config.json",
    repo_id="longluu/Clinical-NER-NCBI-Disease-GatorTronS",
    repo_type="model",
    commit_message="fix the label2id in config"
)

CommitInfo(commit_url='https://huggingface.co/longluu/Clinical-NER-NCBI-Disease-GatorTronS/commit/f87294d962ac365cc8d9c3fb4b7884d44a6bcbb3', commit_message='fix the label2id in config', commit_description='', oid='f87294d962ac365cc8d9c3fb4b7884d44a6bcbb3', pr_url=None, pr_revision=None, pr_num=None)

## Push to model to hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [6]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer

# Load the model
tokenizer = AutoTokenizer.from_pretrained('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/ncbi-disease/gatortrons/')
NER_model = AutoModelForTokenClassification.from_pretrained('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/ncbi-disease/gatortrons/')

# Push the model to hub
NER_model.push_to_hub("longluu/Clinical-NER-NCBI-Disease-GatorTronS", commit_message='--batch_size 24 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01')
tokenizer.push_to_hub("longluu/Clinical-NER-NCBI-Disease-GatorTronS", commit_message='--batch_size 24 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01')

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/longluu/Clinical-NER-NCBI-Disease-GatorTronS/commit/83f96f0b04875ccf18c2a5a80064218eeb61570a', commit_message='--batch_size 24 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01', commit_description='', oid='83f96f0b04875ccf18c2a5a80064218eeb61570a', pr_url=None, pr_revision=None, pr_num=None)